TO-DO LIST:
- [ ] separar training e test set

In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
%matplotlib inline

In [2]:
#df_business=pd.read_feather("../data/business.feather")
#df_review=pd.read_feather("../data/review.feather",columns=['business_id','user_id'])


In [3]:
las_vegas_area=areamanager.delimiter_area(1)
df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'])
df_business=areamanager.pois_in_area(las_vegas_area,df_business)

Area selected: Las Vegas


In [4]:
df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id'])
df_review=pd.merge(df_review,df_business,on='business_id')

In [5]:
#print(df_business.head())
df_review.head()

,user_id,business_id,latitude,longitude,categories
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
1,RBXSJA372ilErzNwz0jXvQ,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
2,x3brMMbJrAW9PwW5A6YL5w,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
3,Skzdl0sWhW88525a1vr59g,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."
4,3Y25VDfnQVcuc33T-U3Z6A,ujmEBvifdJM6h6RLv4wQIg,36.215546,-115.248798,"['Fitness & Instruction', 'Doctors', 'Health &..."


In [6]:
tmp_rec_list=rec.mostpopular(df_review,10)
#df_review=None
tmp_rec_list

,business_id,score
0,2weQS-RnoOBhb1KsHKyoSQ,4534
1,KskYqH1Bi7Z_61pH6Om8pg,4131
2,g8OnV26ywJlZpezdBnOWUQ,3366
3,3kdSl5mo9dWC4clrQjEDGg,3350
4,XXW_OFaYQkkGOGniujZFHg,3330
5,RwMLuOkImBIqqYj4SSKSPg,3283
6,MpmFFw0GE_2iRFPdsRpJbA,3212
7,yfxDa8RFOvJPQh0rNtakHA,3106
8,fL-b760btOaGa85OJ9ut3w,2952
9,t-o_Sraneime4DDhWrQRBA,2879


In [7]:
tmp_rec_list=pd.merge(tmp_rec_list,df_business,on='business_id').drop_duplicates().reset_index(drop=True)
#df_business=None
tmp_rec_list

,business_id,score,latitude,longitude,categories
0,2weQS-RnoOBhb1KsHKyoSQ,4534,36.126887,-115.165590,"['Beauty & Spas', 'Food', 'Event Planning & Se..."
1,KskYqH1Bi7Z_61pH6Om8pg,4131,36.143664,-115.141891,"['Car Dealers', 'Nightlife', 'Automotive', 'Bu..."
2,g8OnV26ywJlZpezdBnOWUQ,3366,36.144713,-115.240092,"['American (New)', 'Breakfast & Brunch', 'Rest..."
3,3kdSl5mo9dWC4clrQjEDGg,3350,36.144085,-115.202486,"['Breakfast & Brunch', 'Sandwiches', 'Burgers'..."
4,XXW_OFaYQkkGOGniujZFHg,3330,36.133667,-115.163095,"['Nightlife', 'Lounges', 'Restaurants', 'Bars'..."
5,RwMLuOkImBIqqYj4SSKSPg,3283,36.131594,-115.164767,"['Mexican', 'Tacos', 'Restaurants']"
6,MpmFFw0GE_2iRFPdsRpJbA,3212,36.128050,-115.164869,"['Lounges', 'Dance Clubs', 'Bars', 'Nightlife']"
7,yfxDa8RFOvJPQh0rNtakHA,3106,36.126194,-115.193445,"['Vietnamese', 'Restaurants', 'Chinese', 'Kore..."
8,fL-b760btOaGa85OJ9ut3w,2952,36.132188,-115.177096,"['Food', 'Burgers', 'Caterers', 'Soul Food', '..."
9,t-o_Sraneime4DDhWrQRBA,2879,36.127684,-115.165905,"['Active Life', 'Hotels & Travel', 'Casinos', ..."


# Categorias relevantes

In [8]:
Lambda = 0.5
Beta = 0.5

In [9]:
df_categories=pd.read_json("../data/categories.json")

# dicionário alias title 2 way
dict_alias_title={}
for index, row in df_categories.iterrows():
    dict_alias_title[row['alias']]=row['title']
    dict_alias_title[row['title']]=row['alias']


category_tree= nx.DiGraph()
for index, row in df_categories.iterrows():
    if not row['parents']:
        category_tree.add_edge(row['alias'],'root') # root node if no parents
    else:
        for parent_label in row['parents']:
            category_tree.add_edge(row['alias'],parent_label)

undirected_category_tree=category_tree.to_undirected()
df_categories=None

In [10]:
nx.shortest_path_length(undirected_category_tree,'aircraftdealers','airsoft')

4

In [11]:
# lista temporária e lista final
# rec_list=pd.DataFrame(columns=['business_id','score'])

# k=len(tmp_rec_list) ## size of rec list
# for i in range(k):
#     print(i)
#     for index,row in tmp_rec_list.iterrows():
#         business_id=row['business_id']
#         score=row['score']
#         print(index,business_id,score)


In [12]:
eval(tmp_rec_list.iloc[0]['categories'])

['Beauty & Spas',
 'Food',
 'Event Planning & Services',
 'Hotels',
 'Buffets',
 'Casinos',
 'Day Spas',
 'Restaurants',
 'Arts & Entertainment',
 'Hotels & Travel']

In [13]:

        
def category_dis_sim(category1,category2):
    dissim=0.0
    spd=nx.shortest_path_length(undirected_category_tree,category1,category2)
    sim = 1.0 / (1.0 + spd)
    dissim=1.0-sim
    return dissim
    

def min_dist_to_list_cat(business,rec_list):
    min_dissim=1.0
    rec_list_size=len(rec_list)
    if rec_list_size==0:
        min_dissim=1.0
    else:
        for index,row in rec_list.iterrows():
            local_min_distance=1
            cur_distance=0
            for category1 in eval(business['categories']):
                for category2 in eval(row['categories']):
                    cur_distance=category_dis_sim(
                        dict_alias_title[category1],
                        dict_alias_title[category2])
                    #print(category1,category2,cur_distance,local_min_distance)
                    local_min_distance=min(local_min_distance,cur_distance)
            min_dissim=min(min_dissim,local_min_distance)
    return min_dissim
        
def objective_ild(business,rec_list):
    # /*(1-_cfg._divWeight) * relevance + */_cfg._divWeight * diversity; where _cfg._divWeight = 0.9
    ret_val= 0.0
    diversity=0.0
    div_weight=0.9
    diversity=min_dist_to_list_cat(business,rec_list)
    ret_val = div_weight*diversity #+ (1-div_weight)*relevance
    return ret_val

min_dist_to_list_cat(tmp_rec_list.iloc[0],tmp_rec_list.iloc[1:9])

0.0

In [14]:
# for index,row in tmp_rec_list.iterrows():
#     business_id=row['business_id']
#     score=row['score']
#     print(index,business_id,score)

In [13]:
#import pandas as pd
#df_review=pd.read_csv('../data/review.csv',usecols=['user_id','business_id'])
#df_business=pd.read_csv('../data/business.csv',usecols=['business_id','categories'])

$\frac{\#RelevantCategoriesInRecList\cap \#RelevantCategoriesToUser}{\#RelevantCategoriesToUser}$

In [15]:

def relevant_categories_to_the_user(user_id,df_review,df_business):
    # focus only on one user
    df=df_review[df_review['user_id']==user_id].copy()
    # number of visits in pois
    df['Count']=df.groupby(['business_id','user_id'])['user_id'].transform('count')
    # merge to get mean poi visits, a column with Count and another with diffpoisvisited
    df_mean_poi_visits=pd.merge(\
    df[['user_id','Count']].groupby('user_id').sum().reset_index(level=0),\
    df[['user_id','business_id']].groupby('user_id').count().reset_index(level=0).rename(columns={"business_id":"diffpoisvisited"}),\
    left_on='user_id',right_on='user_id')
    
    df_mean_poi_visits['meanpoivisits']=df_mean_poi_visits['Count']/df_mean_poi_visits['diffpoisvisited']
    mean_poi_visits=df_mean_poi_visits['meanpoivisits'].iloc[0]
    # print(mean_poi_visits)
    
    # Relevant categories
    relevant_categories = set()    
    for index,row in df.iterrows():
        # Check if poi is relevant
        if row['Count'] > mean_poi_visits:
            # add relevant categories
            for category in eval(df_business[df_business['business_id']==row['business_id']]['categories'].iloc[0]):
                relevant_categories.add(category)
            #print(row['business_id'])
    return relevant_categories
    #print(relevant_pois)
    

def objective_genre_coverage(user_id,business,rec_list,df_review,df_business):
    relevant_categories=relevant_categories_to_the_user(user_id,df_review,df_business)
    count_equal=0
    categories_in_rec_list=set()
    
    for index, row in rec_list.iterrows():
        for category in eval(row['categories']):
            categories_in_rec_list.add(category)
    
    for category in eval(business['categories']):
        categories_in_rec_list.add(category)
    
    #print(categories_in_rec_list)
    
    for cat1 in relevant_categories:
        for cat2 in eval(business['categories']):
            if cat1 == cat2:
                print(cat1)
                count_equal=count_equal+1
    return count_equal/len(relevant_categories)    

print(relevant_categories_to_the_user('---1lKK3aKOuomHnwAkAow',df_review,df_business))
objective_genre_coverage('---1lKK3aKOuomHnwAkAow',tmp_rec_list.iloc[0],tmp_rec_list,df_review,df_business)

{'Nightlife', 'Wine Bars', 'Vegan', 'French', 'Vegetarian', 'Pizza', 'Mexican', 'Italian', 'American (New)', 'Bars', 'Seafood', 'Arts & Entertainment', 'American (Traditional)', 'Wineries', 'Food', 'Cafes', 'Restaurants'}
Arts & Entertainment
Food
Restaurants


0.17647058823529413

# GeoDiv

In [98]:
tmp_rec_list.iloc[1:10]

,business_id,score,latitude,longitude,categories
1,KskYqH1Bi7Z_61pH6Om8pg,4131,36.143664,-115.141891,"['Car Dealers', 'Nightlife', 'Automotive', 'Bu..."
2,g8OnV26ywJlZpezdBnOWUQ,3366,36.144713,-115.240092,"['American (New)', 'Breakfast & Brunch', 'Rest..."
3,3kdSl5mo9dWC4clrQjEDGg,3350,36.144085,-115.202486,"['Breakfast & Brunch', 'Sandwiches', 'Burgers'..."
4,XXW_OFaYQkkGOGniujZFHg,3330,36.133667,-115.163095,"['Nightlife', 'Lounges', 'Restaurants', 'Bars'..."
5,RwMLuOkImBIqqYj4SSKSPg,3283,36.131594,-115.164767,"['Mexican', 'Tacos', 'Restaurants']"
6,MpmFFw0GE_2iRFPdsRpJbA,3212,36.128050,-115.164869,"['Lounges', 'Dance Clubs', 'Bars', 'Nightlife']"
7,yfxDa8RFOvJPQh0rNtakHA,3106,36.126194,-115.193445,"['Vietnamese', 'Restaurants', 'Chinese', 'Kore..."
8,fL-b760btOaGa85OJ9ut3w,2952,36.132188,-115.177096,"['Food', 'Burgers', 'Caterers', 'Soul Food', '..."
9,t-o_Sraneime4DDhWrQRBA,2879,36.127684,-115.165905,"['Active Life', 'Hotels & Travel', 'Casinos', ..."


In [100]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # deg to rad
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2-lon1
    dlat = lat2-lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    r = 6371 # km
    return c*r

In [101]:
df_review['business_id']

0         ujmEBvifdJM6h6RLv4wQIg
1         ujmEBvifdJM6h6RLv4wQIg
2         ujmEBvifdJM6h6RLv4wQIg
3         ujmEBvifdJM6h6RLv4wQIg
4         ujmEBvifdJM6h6RLv4wQIg
                   ...          
891482    lPgE2MIteQVzs39bwfmS6Q
891483    lPgE2MIteQVzs39bwfmS6Q
891484    slA2sb0Ikn76QzBLKYfiRw
891485    slA2sb0Ikn76QzBLKYfiRw
891486    slA2sb0Ikn76QzBLKYfiRw
Name: business_id, Length: 891487, dtype: object

In [102]:
df_review[(df_review['user_id']=='---1lKK3aKOuomHnwAkAow') \
         & (df_review['business_id']==tmp_rec_list.iloc[0]['business_id'])]

,user_id,business_id,latitude,longitude,categories


In [103]:
def poi_distance_from_pois(poi,df_poi):
    return haversine(poi['latitude'],poi['longitude'],df_poi['latitude'],df_poi['longitude'])
poi_distance(tmp_rec_list.iloc[0],df_review[df_review['user_id']=='---1lKK3aKOuomHnwAkAow'])

19158     16.865505
37840     15.849784
38855     13.381600
39074     15.011470
51584      7.947663
            ...    
879428    13.998423
879569     2.056108
880769    10.813656
886249    10.307955
888608    13.552017
Length: 93, dtype: float64

In [109]:
def poi_neighbors(poi,df_poi,distance_km):
    return df_poi[poi_distance_from_pois(poi,df_poi)<distance_km]

poi_neighbors(tmp_rec_list.iloc[0],df_review[df_review['user_id']=='---1lKK3aKOuomHnwAkAow'],1.9)


,user_id,business_id,latitude,longitude,categories
780548,---1lKK3aKOuomHnwAkAow,_cAvID0MXKjD8aveo1Uv0Q,36.131057,-115.173533,"['Health & Medical', 'Massage Therapy', 'Chiro..."


In [127]:


def geo_cov(poi,df_visit_log_in_activity_district,num_rec_pois):
    df_visit_log_in_activity_district=df_visit_log_in_activity_district.copy()
    #df_visit_log_in_activity_district['ac_cover_value']=0
    ac_cover_value_not_distributed=0
    capacity_per_poi=len(df_visit_log_in_activity_district)/num_rec_pois
    neighbors=poi_neighbors(poi,df_visit_log_in_activity_district,2.1).copy()
    neighbors['ac_cover_value']=0
    if len(neighbors)<1:
        print("1")
        ac_cover_value_not_distributed=ac_cover_value_not_distributed+capacity_per_poi
    else:
        print("2")
        capacity_per_neighbor=capacity_per_poi/len(neighbors)
        neighbors['ac_cover_value']=neighbors['ac_cover_value']+capacity_per_neighbor
    
geo_cov(tmp_rec_list.iloc[0],df_review[df_review['user_id']=='---1lKK3aKOuomHnwAkAow'],10)

2
                       user_id             business_id   latitude   longitude  \
501896  ---1lKK3aKOuomHnwAkAow  AxeQEz3-s9_1TyIo-G7UQw  36.153504 -115.151632   
517975  ---1lKK3aKOuomHnwAkAow  XJGMgs9Kh4kcgf8Oskiewg  36.154409 -115.151457   
666945  ---1lKK3aKOuomHnwAkAow  gJd1jOcl5FdqY020Q19n6Q  36.159147 -115.152728   
780548  ---1lKK3aKOuomHnwAkAow  _cAvID0MXKjD8aveo1Uv0Q  36.131057 -115.173533   
879569  ---1lKK3aKOuomHnwAkAow  jGv6tpmE7F9CG5qmLHad6A  36.160085 -115.153644   

                                               categories  ac_cover_value  
501896                ['Barbeque', 'Restaurants', 'Thai']            1.86  
517975  ['Art Galleries', 'Shopping', 'Arts & Entertai...            1.86  
666945  ['Caterers', 'Restaurants', 'Spanish', 'Breakf...            1.86  
780548  ['Health & Medical', 'Massage Therapy', 'Chiro...            1.86  
879569          ['Shopping', 'Costumes', 'Arts & Crafts']            1.86  


0.017453292519943295